In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

def get_coef():
    # Make a coefficient from -4 to 4 
    # excluding -0.4 to 0.4 to avoid the term being irrelevant
    interval_1 = (-4, -0.4)
    interval_2 = (0.4, 4)

    # Randomly choose an interval
    chosen_interval = random.choice([interval_1, interval_2])

    # Generate a random number in the chosen interval
    return round(random.uniform(*chosen_interval), 1)


# Defining base functions
def linear_term(coef, n_points):
    x = np.linspace(-10, 10, n_points)
    return x.reshape(-1, 1), (coef * x).reshape(-1, 1)

def quadratic_term(coef, n_points):
    x = np.linspace(-3, 3, n_points)
    return x.reshape(-1, 1), (coef * x**2).reshape(-1, 1)

def cubic_term(coef, n_points):
    x = np.linspace(-2, 2, n_points)
    return x.reshape(-1, 1), (coef * x**3).reshape(-1, 1)

def quartic_term(coef, n_points):
    x = np.linspace(-2, 2, n_points)
    return x.reshape(-1, 1), (coef * x**4).reshape(-1, 1)

def logarithmic_term(coef, n_points):
    x = np.linspace(0.5, 15, n_points)
    return x.reshape(-1, 1), (coef * np.log(np.abs(x) + 1)).reshape(-1, 1)

def interaction_term(x1, x2, coef):
    # Interaction between two functions f1 and f2
    return coef * x1 * x2

def hinge(coef1, coef2, threshold, n_points):
    x = np.linspace(-10, 10, n_points)
    # Slope changes based on a threshold
    return x.reshape(-1, 1), np.where(x < threshold, coef1*(x-threshold), coef2*(x-threshold)).reshape(-1, 1)

def shuffle_coefficients(X, poly_structure):
    new_structure = []
    y = np.zeros((X.shape[0], 1))
    assert X.shape[1]+1 == len(poly_structure)
    
    
    # Match the polynomial structure of current polynomial
    for i in range(0, len(poly_structure)):
        term_type = poly_structure[i][0]
        if term_type == 'lin':
            coef = get_coef()
            y_new = coef*X[:, i:i+1]
            new_structure.append(('lin', coef))
        elif term_type == 'quad':
            coef = get_coef()
            y_new = coef*(X[:, i:i+1]**2)
            new_structure.append(('quad', coef))
        elif term_type == 'cub':
            coef = get_coef()
            y_new = coef*(X[:, i:i+1]**3)
            new_structure.append(('cub', coef))
        elif term_type == 'hinge':
            coef1 = get_coef()
            coef2 = get_coef()
            threshold = poly_structure[i][1]
            y_new = np.where(X[:, i:i+1] < threshold, coef1*(X[:, i:i+1]-threshold), coef2*(X[:, i:i+1]-threshold))
            new_structure.append(('hinge', threshold, coef1, coef2))
        elif term_type == 'interact':
            coef = round(get_coef()/4, 1)
            y_new = coef*X[:, 0:1]*X[:, 1:2]
            new_structure.append(('interact', coef))
        elif term_type == 'log':
            coef = get_coef()
            y_new = coef*np.log(np.abs(X[:, i:i+1]+1))
            new_structure.append(('log', coef))
        else:
            raise ValueError("Unrecognized term type: ", term_type)
            
        y = y + y_new
        
    return y, new_structure


def conditional_dependency(X, y, poly_structure, n_linear_vars):
    # Random dependency
    rand_var = random.randint(0, n_linear_vars-1) 
    max_val = X[:, rand_var:(rand_var+1)].max()
    min_val = X[:, rand_var:(rand_var+1)].min()
    var_range = max_val - min_val
    max_or_min = random.randint(0, 1)
    
    y_new, new_structure = shuffle_coefficients(X, poly_structure)
    
    if max_or_min == 1:
        # Apply transformation to top 50% of dataset
        y = np.where((X[:, rand_var:rand_var+1] < max_val-0.5*var_range), y, y_new)
        threshold = (rand_var, max_val-0.5*var_range, max_or_min)
    else:
        # Apply transformation to bottom 50% of dataset
        y = np.where((X[:, rand_var:rand_var+1] > min_val+0.5*var_range), y, y_new)
        threshold = (rand_var, max_val-0.5*var_range, max_or_min)
    
    return y, new_structure, threshold


In [ ]:
def cubic_term(coef, n_points):
    x = np.linspace(-2, 2, n_points)
    return x.reshape(-1, 1), (coef * x**3).reshape(-1, 1)

In [ ]:
def generate_synthetic_data(n_points=1000, noise = 2):
    y = np.zeros((n_points, 1))
    X = None
    n_linear_vars = random.randint(3, 5)
    
    poly_structure = []
    
    # Add 3 to 5 linear terms
    for i in range(0, n_linear_vars):
        rand = random.randint(1, 3)
        coef = get_coef()
        
        if rand == 1:
            # add a linear term
            X_new, y_new = linear_term(coef, n_points)
            poly_structure.append(('lin', coef))
        elif rand == 2:
            # add quadratic term
            X_new, y_new = quadratic_term(coef, n_points)
            poly_structure.append(('quad', coef))
        else:
            # rand == 3, add a cubic term
            X_new, y_new = cubic_term(coef, n_points)
            poly_structure.append(('cub', coef))
        
        y = y + y_new
        
        if X is None:
            X = X_new
        else:
            X = np.append(X, X_new, axis = 1)
            
        # Generate a sequence of indices and shuffle them
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)

        # Use the shuffled indices to reorder both arrays
        X = X[indices]
        y = y[indices]

            
        
       
    # Add 1 or 2 hinge terms
    n_hinge_terms = random.randint(1, 2)
    for i in range(0, n_hinge_terms):
        coef1 = get_coef()
        coef2 = get_coef()
        threshold = round(np.random.uniform(-8, 8), 1) # Make a hinge at some point
        X_new, y_new = hinge(coef1, coef2, threshold, n_points)
        X = np.append(X, X_new, axis = 1)
        y = y + y_new
        poly_structure.append(('hinge', threshold, coef1, coef2))
        
        # Generate a sequence of indices and shuffle them
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)

        # Use the shuffled indices to reorder both arrays
        X = X[indices]
        y = y[indices]
    
    
    new_structure = None
    dependancy_threshold = None
    
    # Add a logarithmic term
    coef = get_coef()
    X_new, y_new = logarithmic_term(coef, n_points)
    X = np.append(X, X_new, axis = 1)
    y = y + y_new
    poly_structure.append(('log', coef))
    
    # Generate a sequence of indices and shuffle them
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    # Use the shuffled indices to reorder both arrays
    X = X[indices]
    y = y[indices]
    
    # Add an interaction term with the first two terms (linear/quadratic/cubic terms)
    coef = round(get_coef()/4, 1)
    y_new = interaction_term(X[:, 0:1], X[:, 1:2], coef)
    y = y + y_new
    poly_structure.append(('interact', coef))

    # Add a conditional dependancy to a random variable
    # Take 10% of the variable to create a conditional dependancy
    y, new_structure, dependancy_threshold = conditional_dependency(X, y, poly_structure, n_linear_vars)
    
    # Add intercept and noise
    intercept = round(np.random.uniform(-20, 20), 1)
    y = y + intercept
    y = y + np.random.normal(0, noise, y.shape)

    
    return X, y, poly_structure, new_structure, dependancy_threshold, noise, intercept



In [ ]:
# Function to convert term information into a string representation
def term_to_string(term_type, coeffs):
    if term_type == 'lin':
        return f"{coeffs[1]:.1f}*x{coeffs[0]}"
    elif term_type == 'quad':
        return f"{coeffs[1]:.1f}*x{coeffs[0]}^2"
    elif term_type == 'cub':
        return f"{coeffs[1]:.1f}*x{coeffs[0]}^3"
    elif term_type == 'quart':
        return f"{coeffs[1]:.1f}*x{coeffs[0]}^4"
    elif term_type == 'hinge':
        return f"hinge(x{coeffs[0]}, {coeffs[1]:.1f}, {coeffs[2]:.1f}, {coeffs[3]:.1f})"
    elif term_type == 'interact':
        return f"{coeffs[1]:.1f}*x1*x2"  # Assuming interaction is always between x1 and x2
    elif term_type == 'log':
        return f"{coeffs[1]:.1f}*log(|x{coeffs[0]}| + 1)"
    else:
        return "Unknown term"

# Function to print the polynomial equation
def print_polynomial_equation(poly_structure, new_structure, dependancy_threshold, noise, intercept):
    equation_parts = []
    
    # Add intercept term
    equation_parts.append(str(intercept))
    
    # First part
    dep_var, threshold, max_or_min = dependancy_threshold
    condition = "<=" if max_or_min == 1 else ">="
    dep_string = f"Indicator(x{dep_var+1} {condition} {threshold:.1f})*("
    equation_parts.append(dep_string)

    # Add terms from the original polynomial structure
    for i, term in enumerate(poly_structure):
        term_string = term_to_string(term[0], tuple([i+1])+ term[1:])
        equation_parts.append(term_string)
    
    equation_parts.append(")")
        
    # Conditional dependency part
    dep_var, threshold, max_or_min = dependancy_threshold
    condition = ">" if max_or_min == 1 else "<"
    dep_string = f"Indicator(x{dep_var+1} {condition} {threshold:.1f})*("
    equation_parts.append(dep_string)
        
    # Add terms from the new structure (after conditional dependency)
    for i, term in enumerate(new_structure):
        term_string = term_to_string(term[0], tuple([i+1])+ term[1:])
        equation_parts.append(term_string)
        
    # Close conditional dependency. 
    equation_parts.append(")")
    
    # Add noise term
    equation_parts.append("N(0, " + str(noise) + ")")

    # Combine all parts into a single equation string
    equation = " + ".join(equation_parts)
    print("Polynomial Equation:")
    print(equation)
    

In [ ]:
import pandas as pd
def create_synthetic_data(rs):

    random_seed = rs
    random.seed(random_seed)
    np.random.seed(random_seed)


    X, y, poly_structure, new_structure, dependancy_threshold, noise, intercept = generate_synthetic_data(n_points = 5000, noise = 0)
    print_polynomial_equation(poly_structure, new_structure, dependancy_threshold, noise, intercept)
    true = np.append(y, X, axis = 1)
    true = pd.DataFrame(true)
    new_column_names = ['y']
    new_column_names.extend([f'x{i}' for i in range(1, len(true.columns))])
    true.columns = new_column_names
    true = true.reset_index(drop = True)
    
    
    y = y + np.random.normal(0, 10, y.shape)
    noised = np.append(y, X, axis = 1)
    noised = pd.DataFrame(noised)
    new_column_names = ['y']
    new_column_names.extend([f'x{i}' for i in range(1, len(noised.columns))])
    noised.columns = new_column_names
    
    # Generate a sequence of indices and shuffle them
    noised = noised.reset_index(drop = True)
    
    return noised, true

In [ ]:
train1, test1 = create_synthetic_data(1)
train2, test2 = create_synthetic_data(2)
train3, test3 = create_synthetic_data(3)
train4, test4 = create_synthetic_data(4)
train5, test5 = create_synthetic_data(5)


In [ ]:
train1.to_csv("Test Data/Synthetic/rs1_new_train.csv", index = False)
test1.to_csv("Test Data/Synthetic/rs1_new_test.csv", index = False)

train2.to_csv("Test Data/Synthetic/rs2_new_train.csv", index = False)
test2.to_csv("Test Data/Synthetic/rs2_new_test.csv", index = False)

train3.to_csv("Test Data/Synthetic/rs3_new_train.csv", index = False)
test3.to_csv("Test Data/Synthetic/rs3_new_test.csv", index = False)

train4.to_csv("Test Data/Synthetic/rs4_new_train.csv", index = False)
test4.to_csv("Test Data/Synthetic/rs4_new_test.csv", index = False)

train5.to_csv("Test Data/Synthetic/rs5_new_train.csv", index = False)
test5.to_csv("Test Data/Synthetic/rs5_new_test.csv", index = False)


